**Step 01:** Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, r2_score

**Step 02:** Get the transect besed data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive'), force_remount = True

In [ ]:
data = pd.read_csv ('/content/drive/My Drive/Transects.csv')

**Step 03:** Calculate product of clay and sand percentage

In [ ]:
data['Smeans'] = (data['S0'] + data['S10'] + data['S30'] + data['S60'] + data['S100'] + data['S200']) / 6
data['Cmeans'] = (data['C0'] + data['C10'] + data['C30'] + data['C60'] + data['C100'] + data['C200']) / 6
data['Pmeans'] = (data['Cmeans']*data['Smeans'])

**Step 04:** Categorize overall dataset based on LR2

In [ ]:
df = pd.DataFrame(data)
df1 = pd.DataFrame(data[data['LR2']>=0.1])
df2 = pd.DataFrame(data[data['LR2']>=0.2])
df3 = pd.DataFrame(data[data['LR2']>=0.3])
df4 = pd.DataFrame(data[data['LR2']>=0.4])
df5 = pd.DataFrame(data[data['LR2']>=0.5])
df6 = pd.DataFrame(data[data['LR2']>=0.6])
df7 = pd.DataFrame(data[data['LR2']>=0.7])
df8 = pd.DataFrame(data[data['LR2']>=0.8])
df9 = pd.DataFrame(data[data['LR2']>=0.9])

**Step 05:** Divide each dataset into erosional and accretional data based on LRR

In [ ]:
dataE = df[df['LRR'] < 0]
dataD = df[df['LRR'] > 0]
dataE1 = df1[df1['LRR'] < 0]
dataD1 = df1[df1['LRR'] > 0]
dataE2 = df2[df2['LRR'] < 0]
dataD2 = df2[df2['LRR'] > 0]
dataE3 = df3[df3['LRR'] < 0]
dataD3 = df3[df3['LRR'] > 0]
dataE4 = df4[df4['LRR'] < 0]
dataD4 = df4[df4['LRR'] > 0]
dataE5 = df5[df5['LRR'] < 0]
dataD5 = df5[df5['LRR'] > 0]
dataE6 = df6[df6['LRR'] < 0]
dataD6 = df6[df6['LRR'] > 0]
dataE7 = df7[df7['LRR'] < 0]
dataD7 = df7[df7['LRR'] > 0]
dataE8 = df8[df8['LRR'] < 0]
dataD8 = df8[df8['LRR'] > 0]
dataE9 = df9[df9['LRR'] < 0]
dataD9 = df9[df9['LRR'] > 0]

**Step 06:** Make machine learning models

In [ ]:
rf = RandomForestRegressor(random_state = 42)
gb = GradientBoostingRegressor(random_state = 42)

**Step 07:** For all datasets (Random Forest)
*   Designate x and y varibales
*   Split both x and y into training and testing datasets
*   Nornalize the dataset
*   Apply cross-validation and calculate R2 score and RMSE
*   Train the model and predict output
*   Calculate R2 score and RMSE for testing datasets
*   Calculate feature importance score.

In [ ]:
X = data[['Orientation', 'Rmeans', 'Width', 'Dist_sea', 'Slope', 'Dist_SS']] # Change the data each time
y = data['LRR']                                                              # Change the data each time
X_train, X_test, y_train, y_rfT0_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

cv_results = cross_validate(rf, X_train, y_train, cv = 5, n_jobs = -1, verbose = 2,
                            scoring = {'r2': 'r2', 'mse':'neg_mean_squared_error'})

r2V_Trf_LR20 = np.mean(cv_results['test_r2'])
rmseV_Trf_LR20 = np.sqrt(np.abs(np.mean(cv_results['test_mse'])))
print(f'R2 (Validation): {r2V_Trf_LR20}')
print(f'RMSE (Validation): {rmseV_Trf_LR20}')

rfT0 = rf.fit (X_train, y_train)
y_rfT0 = rfT0.predict (X_test)

r2T_Trf_LR20 = r2_score (y_rfT0_test, y_rfT0)
rmseT_Trf_LR20 = np.sqrt(mean_squared_error (y_rfT0_test, y_rfT0))
print ('R2 (Testing):', r2T_Trf_LR20)
print ('RMSE (Testing):', rmseT_Trf_LR20)
print ('Feature Importance:', rfT0.feature_importances_)

**Step 07:** For all datasets (Gradient Boosting)
*   Designate x and y varibales
*   Split both x and y into training and testing datasets
*   Nornalize the dataset
*   Apply cross-validation and calculate R2 score and RMSE
*   Train the model and predict output
*   Calculate R2 score and RMSE for testing datasets
*   Calculate feature importance score.

In [ ]:
X = data[['Orientation', 'Rmeans', 'Width_Mod', 'Dist_sea', 'Slope', 'Dist_SS']] # Change the data each time
y = data['LRR']                                                                  # Change the data each time
X_train, X_test, y_train, y_gbT0_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

cv_results = cross_validate(gb, X_train, y_train, cv = 5, n_jobs = -1, verbose = 2, scoring = {'r2': 'r2', 'mse':'neg_mean_squared_error'})

r2V_Tgb_LR20 = np.mean(cv_results['test_r2'])
rmseV_Tgb_LR20 = np.sqrt(np.abs(np.mean(cv_results['test_mse'])))
print(f'R2 (Validation): {r2V_Tgb_LR20}')
print(f'RMSE (Validation): {rmseV_Tgb_LR20}')

gbT0 = gb.fit (X_train, y_train)
y_gbT0 = gbT0.predict (X_test)

r2T_Tgb_LR20 = r2_score (y_gbT0_test, y_gbT0)
rmseT_Tgb_LR20 = np.sqrt(mean_squared_error (y_gbT0_test, y_gbT0))
print ('R2 (Testing):', r2T_Tgb_LR20)
print ('RMSE (Testing):', rmseT_Tgb_LR20)
print ('Feature Importance:', gbT0.feature_importances_)